In [11]:
class IO():
    def __init__(self, input_=[]):
        self.input_buffer = input_.copy()
        self.output_buffer = []
    
    def add_input(self, value, map_fn=int):
        for c in value:
            self.input_buffer.append(map_fn(c))
        return self

    def read(self):
        return self.input_buffer.pop(0)
    
    def write(self, value):
        self.output_buffer.append(value)
        return self
    
    def output(self):
        return self.output_buffer

In [12]:
import advent
from advent.intcode import run
import numpy as np

data = advent.get_intcode(19)

res = np.full((50, 50), '.')
print_np = lambda res: print('\n'.join([''.join(line) for line in res]))

count = 0
for i in range(50):
    for j in range(50):
        _, out = run(data.copy(), IO([i, j]))
        if out[0] == 1:
            count += 1
        res[(i, j)] = ('#' if out[0] else '.')

print(count)

186


In [33]:
def getter(x, y, data):
    if x < 0 or y < 0: return 0
    _, out = run(data.copy(), IO([x, y]))
    return out[0]

def attempt_grid(getter, size=3, y=5):
    # on the line with coordinates (x+size, :),
    # attempts to find the first '#'
    # then from (x,y) and see if you find enough '#' to complete the square
    # e.g. attempt_grid(3, 20) = False, attempt_grid(3, 21) = True
    x = 0
    while True:
        if getter(x, y+size-1) == 1: break
        x += 1
    res = getter(x+size-1, y) == 1 and getter(x+size-1, y+size-1) == 1 and getter(x, y) == 1
    print(f"Now attempting size={size}, y={y}, found x={x}, size={size}, xy={str(x*10000 + y)}. Result: {res}")
    #print(f"Corners: {(x, y)}:{getter(x, y)}, {(x+size-1, y)}:{getter(x+size-1, y)}, {(x, y+size-1)}:{getter(x, y+size-1)}, {(x+size-1, y+size-1)}:{getter(x+size-1, y+size-1)}")
    return res

def binary_search(getter, size = 100, min_ = 10, max_ = 1200):
    # Find the minimum x for which attempt_grid(size, x) = True
    # assume attempt_grid(min_, x) = False and attempt_grid(max_, x) = True
    if (max_ - min_) <= 1:
        assert attempt_grid(getter, size, max_)
        return max_
    next = (min_ + max_) // 2
    if attempt_grid(getter, size, next):
        return binary_search(getter, size, min_, next)
    return binary_search(getter, size, next, max_)


In [36]:
max_ = binary_search(lambda x, y: getter(x, y, data.copy()), 100)
# HOWEVER this wasn't the correct answer
# I then found out that the assumption of binary search isn't actually true:
# There is a solution lower (but close)

Now attempting size=100, y=605, found x=524, size=100, xy=5240605. Result: False
Now attempting size=100, y=902, found x=745, size=100, xy=7450902. Result: False
Now attempting size=100, y=1051, found x=856, size=100, xy=8561051. Result: False
Now attempting size=100, y=1125, found x=911, size=100, xy=9111125. Result: False
Now attempting size=100, y=1162, found x=939, size=100, xy=9391162. Result: True
Now attempting size=100, y=1143, found x=925, size=100, xy=9251143. Result: False
Now attempting size=100, y=1152, found x=932, size=100, xy=9321152. Result: True
Now attempting size=100, y=1147, found x=928, size=100, xy=9281147. Result: True
Now attempting size=100, y=1145, found x=926, size=100, xy=9261145. Result: True
Now attempting size=100, y=1144, found x=926, size=100, xy=9261144. Result: False
Now attempting size=100, y=1145, found x=926, size=100, xy=9261145. Result: True


In [39]:
for y in range(max_ - 5, max_):
    attempt_grid(lambda x, y: getter(x, y, data.copy()), 100, y)

Now attempting size=100, y=1140, found x=923, size=100, xy=9231140. Result: False
Now attempting size=100, y=1141, found x=923, size=100, xy=9231141. Result: True
Now attempting size=100, y=1142, found x=924, size=100, xy=9241142. Result: False
Now attempting size=100, y=1143, found x=925, size=100, xy=9251143. Result: False
Now attempting size=100, y=1144, found x=926, size=100, xy=9261144. Result: False


In [35]:
# This is what I used for debugging, in combination with print_np(res) to see when the first grid of size 3 would occur
binary_search(lambda x, y: getter(x, y, data.copy()), 3, min_=15, max_=35)

Now attempting size=3, y=25, found x=21, size=3, xy=210025. Result: False
Now attempting size=3, y=30, found x=24, size=3, xy=240030. Result: True
Now attempting size=3, y=27, found x=22, size=3, xy=220027. Result: True
Now attempting size=3, y=26, found x=21, size=3, xy=210026. Result: True
Now attempting size=3, y=26, found x=21, size=3, xy=210026. Result: True


26